In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/KoELECTRA/finetune

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/KoELECTRA/finetune


In [ ]:
!pip install --upgrade transformers
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached seqeval-1.2.2.tar.gz (43 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=9c2b8e6523d15018ef25c15840270970bec2c1802f11656e824275543abfc592
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
import argparse
import json
import logging
import os
import glob
import re

import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from fastprogress.fastprogress import master_bar, progress_bar

from transformers import (
    AdamW,
    get_linear_schedule_with_warmup
)

from src import (
    CONFIG_CLASSES,
    TOKENIZER_CLASSES,
    MODEL_FOR_SEQUENCE_CLASSIFICATION,
    init_logger,
    set_seed,
    compute_metrics
)
from processor import seq_cls_load_and_cache_examples as load_and_cache_examples
from processor import seq_cls_tasks_num_labels as tasks_num_labels
from processor import seq_cls_processors as processors
from processor import seq_cls_output_modes as output_modes

logger = logging.getLogger(__name__)


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "Haaaaeun/koELECTRA_hatespeech"

# Load the tokenizer and model
tokenizer_v1 = AutoTokenizer.from_pretrained(model_name)
model_v1 = AutoModelForSequenceClassification.from_pretrained(model_name)


In [ ]:
model_name = "Haaaaeun/koELECTRA_hatespeech_v2"

# Load the tokenizer and model
tokenizer_v2 = AutoTokenizer.from_pretrained(model_name)
model_v2 = AutoModelForSequenceClassification.from_pretrained(model_name)


In [ ]:

# Set the device (CPU or GPU) for inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_v1.to(device)
model_v2.to(device)


In [ ]:

# Example input text
input_text = "딱 봐도 모르냐 ㅂㅅ."


In [ ]:
# Tokenize the input text
inputs_v1 = tokenizer_v1.encode_plus(input_text, padding=True, truncation=True, return_tensors="pt")
inputs_v1 = inputs_v1.to(device)

# Perform inference
model_v1.eval()
with torch.no_grad():
    outputs_v1 = model_v1(**inputs_v1)

# Process the outputs
logits_v1 = outputs_v1.logits
probabilities_v1 = torch.softmax(logits_v1, dim=1)
prediction_v1 = torch.argmax(probabilities_v1, dim=1).item()

# Print the prediction
print("Prediction:", prediction_v1)


In [ ]:
# Tokenize the input text
inputs_v2 = tokenizer_v2.encode_plus(input_text, padding=True, truncation=True, return_tensors="pt")
inputs_v2 = inputs_v2.to(device)

# Perform inference
model_v2.eval()
with torch.no_grad():
    outputs_v2 = model_v2(**inputs_v2)

# Process the outputs
logits_v2 = outputs_v2.logits
probabilities_v2 = torch.softmax(logits_v2, dim=1)
prediction_v2 = torch.argmax(probabilities_v2, dim=1).item()

# Print the prediction
print("Prediction:", prediction_v2)
